In [1]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [12]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [13]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=315
end_page_index=479
pdf_parsed_output_dir = '../pdf_parsing/parsed_pdf_output_pickle/'
pdf_enrichment_output_dir = './pdf_enriched_output/'
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename

In [14]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.2)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [ ]:
#Ininitial Load From Parsed Content (deserialized data)
with open(pdf_parsed_output_dir+'pdf_content_dict_stage1.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)
document_dict_deserialized=document_dict_deserialized[87:566]

###Intermediate Load in case of errors
#with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_entity_extraction_314.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)

In [15]:
len(document_dict_deserialized)

479

In [16]:
##Definitions of Individual Enrichment Modules######

# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string

def extract_entities_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page related to a particular technology and extract \n
            the main entities or objects from the document.The entities or objects can be anything surrounding \n
            technology in which your expetise lies.By Objects or entities i mean it can be a programming Language,\n
            command,framework,database,process,cache,controller,qeueues,schedulers,errors,exceptions or any \n
            components related to technology. \n
            Never extract page number or any kind of number as entity. \n
            surrounding the technology.Please output in form of list of json where each extracted entity will be \n
            represented by a json along with description and category.\n
            The category name can be like \n
            hardware,software,network,application,database,process,thread,container etc.Mention the category name \n
            in third bracket. \n
            After extracting all the entities along with their description and category format as per rules below: \n
            1.For each entity create a json with 3 keys "entity","description" and "category". \n
            2.Each extracted entity must be enlosed within double quotes. \n
            3.There should not be any unterimnated string for "entity","description" and "category".In case it is \n
            unterminated due to any reason put the ending double quote to it.\n
            3.Each extracted entity must be a valid word,phrase or a complete command.\n
            4.Extracted entity should never be a character,part of a word,alphabet,number or any alphaneumeric.\n
            After all the entities of the page are extracted where each entity is a json then only collate all the entity json \m
            into a list of json.
            5.If any of the extracted entity,description value contain character ':',handle it as a part of entity \n
            or decription ,dont use the character ':' found in entity or description as  a delimiter.\n
            Output the collated list of json where each entity is represented by json. \n
            Do not output anything other than the list of json neither heading/decsription before the list of json\n
            nor any decsription/footer below the json \n
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []
    
    #print("Page Text")
    #print(page_text)

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+json_output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
    
    return json_output
    
    #return page_output_json
    #return output

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


In [20]:
######This code enrcihes the PDF page by page and while doing so it fails after processing 20 to 50 pages####
######currently we are manually tackling this problem by making the pdf dict list into aglobal variable######
######After the llm fails we take the index of the last page done and save the pdf dict list uptil that ####
#####Then as of now i am manually restarting the by loading the saved pdf dict list as the source pdf and ###
#####Using the last done pdf page index +1 as the starting index.So (last Done page Index + 1) is the ######
###starting index###
####The maximum page has been given as 565 as that is the last page where the main content of the document###
####resides.In case of the book Kubernetes in action it is 565.##############################################
####If this is batch process then the last page can be manually seen and used as the variable else one can ####
#####also go for the pdf enrichment of all the pages of the book#############################################

###This manual thing needs to be automated by either langraph or by using agents framework##################

def enrich_page(page_idx):
    
    print("Page Number")
    print(page_idx)
    
    page_text=document_dict_deserialized[page_idx]['text']
    
    ##Entity Extraction Enrichment
    page_entity_lst_dict=extract_entities_per_page(page_text)
    
    #print("Page Entity List Dictionary")
    #print(page_entity_lst_dict)
    
    print("Page Entity List Dictionary Cleaned")
    print(page_entity_lst_dict)
    
    try:
        page_entity_lst_json=json.loads(page_entity_lst_dict)
        document_dict_deserialized[page_idx]['entities']=page_entity_lst_json
        print("Done for page "+str(page_idx))
        return page_idx
    except:
        print("Failed for page "+str(page_idx))
        return page_idx-1
        
    ##Collate entity list per page into unique Entities
    
    #entity_lst=entity_collector_per_page(page_entity_lst_json)
    
    ##Relationship Extraction Enrichment
    #page_relationship_lst_dict=extract_relationship_per_page(page_text,entity_lst)
    
    #print("Page Relationship List Dictionary")
    #print(page_entity_lst_dict)
    
    #document_dict_deserialized[page_idx]['relationships']=page_relationship_lst_dict
    print("Done for page "+str(page_idx))
    return page_idx

In [21]:

for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        
        
        
        
    

Page Number
315
Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "", "category": "software"},
  {"entity": "API server", "description": "", "category": "application"},
  {"entity": "ClusterRoles", "description": "predefined roles for Kubernetes", "category": "software"},
  {"entity": "view ClusterRole", "description": "allows read-only access to resources", "category": "software"},
  {"entity": "edit ClusterRole", "description": "allows modifying resources in a namespace", "category": "software"},
  {"entity": "admin ClusterRole", "description": "grants complete control of resources in a namespace", "category": "software"},
  {"entity": "cluster-admin ClusterRole", "description": "grants complete control of the Kubernetes cluster", "category": "software"},
  {"entity": "ServiceAccounts", "description": "accounts used by user-defined pods", "category": "software"},
  {"entity": "Roles", "description": "predefined roles for Kubernetes", "category": "softwar

Page Entity List Dictionary Cleaned
[{"entity": "hostPort", "description": "a feature to expose system services deployed to every node using DaemonSets", "category": "container"}, {"entity": "kubia-hostport", "description": "a pod's YAML definition", "category": "pod"}, {"entity": "apiVersion", "description": "a field in the YAML definition", "category": "field"}, {"entity": "v1", "description": "the version of the API", "category": "API version"}, {"entity": "kind", "description": "a field in the YAML definition", "category": "field"}, {"entity": "Pod", "description": "a type of Kubernetes object", "category": "object"}, {"entity": "metadata", "description": "a section in the YAML definition", "category": "section"}, {"entity": "name", "description": "a field in the metadata section", "category": "field"}, {"entity": "kubia-hostport", "description": "the name of the pod", "category": "pod"}, {"entity": "spec", "description": "a section in the YAML definition", "category": "section"}, 

Page Entity List Dictionary Cleaned
[
  {"entity": "kube-proxy", "description": "a pod that needs to modify the node's iptables rules to make services work", "category": "application"},
  {"entity": "kubeadm", "description": "a tool for deploying a cluster", "category": "software"},
  {"entity": "privileged", "description": "a property in the container's security-Context property set to true", "category": "property"},
  {"entity": "apiVersion", "description": "a field in the YAML specification of a pod", "category": "field"},
  {"entity": "/dev", "description": "a special file directory on Linux that contains device files for all devices on the system", "category": "directory"},
  {"entity": "device files", "description": "special files used to communicate with devices", "category": "files"},
  {"entity": "kubectl", "description": "a command-line tool for interacting with a Kubernetes cluster", "category": "command"},
  {"entity": "exec", "description": "a subcommand of kubectl that al

Page Entity List Dictionary Cleaned
[{"entity": "container", "description": "a lightweight and standalone executable software package", "category": "software"}, 
 {"entity": "security context", "description": "a set of options that define how a container runs", "category": "software"}, 
 {"entity": "spec", "description": "a configuration option for the container", "category": "software"}, 
 {"entity": "containers", "description": "a list of containers in the pod", "category": "software"}, 
 {"entity": "image", "description": "the base image used to create a container", "category": "software"}, 
 {"entity": "command", "description": "an instruction that is executed when the container starts", "category": "software"}, 
 {"entity": "securityContext", "description": "a configuration option for the container's security context", "category": "software"}, 
 {"entity": "readOnlyRootFilesystem", "description": "a boolean option that makes the root filesystem read-only", "category": "software"},

Page Entity List Dictionary Cleaned
[
  {"entity": "kernel capabilities", "description": "", "category": "software"},
  {"entity": "SELinux labels", "description": "", "category": "software"},
  {"entity": "writable root filesystem", "description": "", "category": "hardware"},
  {"entity": "filesystem groups", "description": "", "category": "hardware"},
  {"entity": "volume types", "description": "", "category": "hardware"},
  {"entity": "PodSecurityPolicy", "description": "", "category": "software"},
  {"entity": "host's IPC", "description": "", "category": "hardware"},
  {"entity": "host's PID", "description": "", "category": "hardware"},
  {"entity": "host's Network namespace", "description": "", "category": "hardware"},
  {"entity": "privileged containers", "description": "", "category": "software"},
  {"entity": "host ports", "description": "", "category": "hardware"},
  {"entity": "10000-11000", "description": "", "category": ""},
  {"entity": "13000-14000", "description": "", "c

...

Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "", "category": "software"},
  {"entity": "API server", "description": "", "category": "software"},
  {"entity": "kubectl", "description": "", "category": "command"},
  {"entity": "get", "description": "", "category": "command"},
  {"entity": "websites", "description": "", "category": "resource"},
  {"entity": "kubia", "description": "", "category": "resource"},
  {"entity": "Website.v1.extensions.example.com", "description": "", "category": "resource"},
  {"entity": "extensions.example.com/v1", "description": "", "category": "API"},
  {"entity": "apiVersion", "description": "", "category": "field"},
  {"entity": "kind", "description": "", "category": "field"},
  {"entity": "metadata", "description": "", "category": "resource"},
  {"entity": "creationTimestamp", "description": "", "category": "field"},
  {"entity": "name", "description": "", "category": "field"},
  {"entity": "namespace", "description": "",

Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "", "category": "software"},
  {"entity": "API server", "description": "", "category": "software"},
  {"entity": "controller", "description": "", "category": "software"},
  {"entity": "Service", "description": "", "category": "software"},
  {"entity": "Deployment", "description": "", "category": "software"},
  {"entity": "Pod", "description": "", "category": "software"},
  {"entity": "kubectl", "description": "", "category": "software"},
  {"entity": "ADDED event", "description": "", "category": "event"},
  {"entity": "201 Created response", "description": "", "category": "response"},
  {"entity": "Deployment, Service and Pods", "description": "", "category": "listing"},
  {"entity": "kubia-website Website", "description": "", "category": "software"},
  {"entity": "Service objects", "description": "", "category": "software"},
  {"entity": "node port", "description": "", "category": "port"},
  {"entity": "st

Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "", "category": "software"},
  {"entity": "Service Catalog", "description": "", "category": "software"},
  {"entity": "ClusterServiceClass", "description": "a class of services offered by a broker", "category": "software"},
  {"entity": "postgres-database", "description": "a ClusterServiceClass for PostgreSQL databases", "category": "software"},
  {"entity": "free", "description": "a plan name for the postgres-database ClusterServiceClass", "category": "software"},
  {"entity": "ServiceInstance", "description": "a resource that represents a provisioned service", "category": "software"},
  {"entity": "parameters", "description": "additional information passed to the broker when provisioning a service", "category": "software"},
  {"entity": "init-db-args", "description": "a parameter specific to the postgres-database ClusterServiceClass", "category": "software"},
  {"entity": "--data-checksums", "description"

Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "", "category": "software"},
  {"entity": "OpenShift", "description": "", "category": "software"},
  {"entity": "template", "description": "a processed template that can be created with a single POST request", "category": "application"},
  {"entity": "parameters", "description": "values for the template's parameters", "category": "process"},
  {"entity": "POST request", "description": "", "category": "command"},
  {"entity": "Java EE application", "description": "", "category": "application"},
  {"entity": "Application Server", "description": "", "category": "software"},
  {"entity": "back-end database", "description": "", "category": "database"},
  {"entity": "BuildConfig", "description": "a resource that triggers builds of container images immediately after a change is committed to the source Git repository", "category": "process"},
  {"entity": "Git repository", "description": "", "category": "software"}

Page Entity List Dictionary Cleaned
[
  {"entity": "kubectl", "description": "", "category": "software"},
  {"entity": "minikube", "description": "", "category": "software"},
  {"entity": "Google Kubernetes Engine", "description": "", "category": "hardware"},
  {"entity": "kubeconfig", "description": "", "category": "process"},
  {"entity": "kubectl", "description": "", "category": "software"},
  {"entity": "GKE cluster", "description": "", "category": "database"},
  {"entity": "minikube start", "description": "", "category": "command"},
  {"entity": "gcloud container clusters get-credentials", "description": "", "category": "command"}
]
Done for page 477
Page Number
478
Page Entity List Dictionary Cleaned
[
  {"entity": "kubectl", "description": "command-line tool for interacting with Kubernetes clusters", "category": "software"},
  {"entity": "clusters", "description": "list of available Kubernetes clusters", "category": "database"},
  {"entity": "namespaces", "description": "logical

In [10]:
def entity_extractor(start_idx,end_idx):
    for idx in range(start_idx,start_idx):
        enrich_page(idx)

{'page': 556,
 'img_cnt': 0,
 'img_flag': 0,
 'img_npy_lst': [],
 'text': '524\nCHAPTER 18\nExtending Kubernetes\n18.2.4 Provisioning and using a service\nLet’s imagine the pods you’re deploying need to use a database. You’ve inspected the\nlist of available ClusterServiceClasses and have chosen to use the free plan of the\npostgres-database ClusterServiceClass. \nPROVISIONING A SERVICEINSTANCE\nTo have the database provisioned for you, all you need to do is create a Service-\nInstance resource, as shown in the following listing.\napiVersion: servicecatalog.k8s.io/v1alpha1\nkind: ServiceInstance\nmetadata:\n  name: my-postgres-db                     \nspec:\n  clusterServiceClassName: postgres-database        \n  clusterServicePlanName: free                             \n  parameters:\n    init-db-args: --data-checksums         \nYou created a ServiceInstance called my-postgres-db (that will be the name of the\nresource you’re deploying) and specified the ClusterServiceClass and the ch

In [22]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_phase1_entity_extraction_478_final.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
document_dict_deserialized[155]

{'page': 244,
 'img_cnt': 0,
 'img_flag': 0,
 'img_npy_lst': [],
 'text': '212\nCHAPTER 7\nConfigMaps and Secrets: configuring applications\nEDITING A CONFIGMAP\nLet’s see how you can change a ConfigMap and have the process running in the pod\nreload the files exposed in the configMap volume. You’ll modify the Nginx config file\nfrom your previous example and make Nginx use the new config without restarting\nthe pod. Try switching gzip compression off by editing the fortune-config Config-\nMap with kubectl edit:\n$ kubectl edit configmap fortune-config\nOnce your editor opens, change the gzip on line to gzip off, save the file, and then\nclose the editor. The ConfigMap is then updated, and soon afterward, the actual file\nin the volume is updated as well. You can confirm this by printing the contents of the\nfile with kubectl exec:\n$ kubectl exec fortune-configmap-volume -c web-server\n➥  cat /etc/nginx/conf.d/my-nginx-config.conf\nIf you don’t see the update yet, wait a while and try

In [ ]:
####The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####

In [48]:
# Load data (deserialize)
with open('pdf_content_dict_page_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)